In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
torch.backends.mps.is_available()
torch.backends.mps.is_built()
mps_device = torch.device("mps")
x = torch.ones(45, device="mps")

In [31]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [38]:
input: str | list = "hello world"
tokenized_input = tokenizer.encode(input, return_tensors='pt')

In [44]:
output = model(tokenized_input)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.1909,  -7.0791,  -7.0640,  ...,  -6.1842,  -6.2977,  -4.2038],
         [ -9.6314,  -9.4228,  -9.5366,  ...,  -9.2170,  -8.3938,  -5.9639],
         [-12.5930, -12.9342, -12.8723,  ..., -11.7490, -11.0491,  -4.2497],
         [-12.2999, -12.2242, -12.0449,  ..., -10.0910,  -9.9382, -11.6595]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [45]:
masked_token_index = torch.where(tokenized_input == tokenizer.mask_token_id)[1]
predicted_token_ids = torch.argmax(output[0][0, masked_token_index], dim=1)

In [47]:
predicted_text = tokenizer.decode(predicted_token_ids)

print(predicted_text)